# Analyze Similar Pairs
This small project will complete jobs below:
1. Get all the similar pairs that output from LSH and SetJoin
2. Analyze them (Know how many documents among them, verify if there is any false positive)
3. Compare them (Is there any inclusion relationship)


## Load Similar Pairs

In [6]:
import tqdm
from glob import glob
import argparse

# load from LSH

dup_dir_path = "/research/projects/zp128/RedPajama-Data-1T/RedPajama-Data-1T/RedPajama_norm/dup"
files = glob(f"{dup_dir_path}/*")
simp_lsh = set()
for fp in files:
    with open(fp, "r") as f:
        for line in tqdm.tqdm(f):
            ori_pair = tuple(line.strip().split(" :: "))
            pair = tuple([part.split("@")[1] for part in ori_pair])
            if pair[0] != pair[1]:
                simp_lsh.add(pair)




7349it [00:00, 226781.69it/s]
7223it [00:00, 212022.41it/s]
7405it [00:00, 217806.85it/s]
7430it [00:00, 224612.80it/s]
7314it [00:00, 178313.99it/s]
7316it [00:00, 311759.25it/s]
7234it [00:00, 215721.04it/s]
7317it [00:00, 211513.22it/s]
7458it [00:00, 218796.52it/s]


In [1]:
# load from SetJoin
import util
simpairs_bin_path = "/research/projects/zp128/RedPajama_Analysis/SetJoin/similar_pairs/stackexchange_sim_pairs_0.800000.bin"
idmap_bin_path = "/research/projects/zp128/RedPajama_Analysis/SetJoin/sorted_sets/stackexchange_idmap.bin"

idmap = util.read_ints_from_binary(idmap_bin_path)
sim_pairs = util.read_pairs_from_binary(simpairs_bin_path)
simp_setjoin = util.map_elements(sim_pairs, idmap)

simp_setjoin = correct_pair_order(simp_setjoin)
simp_lsh = correct_pair_order(simp_lsh)

## Analyze them

In [ ]:
# load the real documents



## Compare them

In [11]:
union_set = simp_setjoin.union(simp_lsh)
union_size = len(union_set)
print(f"The union of two sets includes {union_size} unique pairs.")

# Intersection
intersection_set = simp_setjoin.intersection(simp_lsh)
intersection_size = len(intersection_set)
print(f"The intersection of two sets includes {intersection_size} common pairs.")

# Difference
difference_set = simp_setjoin.difference(simp_lsh)  # B - A
difference_size = len(difference_set)
print(f"The difference  of two sets(B - A) includes {difference_size} common pairs.")


The union of two sets includes 49339 unique pairs.
The intersection of two sets includes 0 common pairs.
The intersection of two sets includes 31950 common pairs.
